In [49]:
import torch
from datasets import load_dataset
import torch.nn as nn
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import timm
from urllib.request import urlopen
from PIL import Image

In [50]:
query_path = 'query_images/'
query_files = os.listdir(query_path)
print(len(query_files))

retrieval_path = 'images/'
retrieval_files = os.listdir(retrieval_path)
print(len(retrieval_files))

print(query_files[:5])

200
10000
['64d323101c620faef16a9afd_6_agstore_vn.jpg\n', '64da9bac1c3e4f08a26d57d8_2_chicland_vn.jpg\n', '64d25998c0c243fd215f711f_0_hhluxury_vn.jpg\n', '64da8ec21c3e4f08a26d43a6_1_chicland_vn.jpg\n', '64d3238a1c620faef16a9beb_5_agstore_vn.jpg\n']


In [51]:
model = timm.create_model(
            'eva02_base_patch14_448.mim_in22k_ft_in22k_in1k',
            pretrained=True,
            num_classes=0,  # remove classifier nn.Linear
        )
model = model.eval()
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

In [4]:
# n_queries = len(query_files)
# query_matrix = np.zeros((n_queries, 768))
# for i in tqdm(range(n_queries)):
#     img = Image.open(query_path+query_files[i]).convert('RGB')
#     vector = model(transforms(img).unsqueeze(0)).detach().numpy()
#     query_matrix[i, :] = vector    
# print(query_matrix.shape)
# print(query_matrix)

In [38]:
# np.save('EVA_query_matrix.npy', query_matrix)
# np.save('query_labels.npy', query_files)

In [6]:
# loaded_matrix = np.load('EVA_query_matrix.npy')
# print(loaded_matrix)
# list_query_label = np.load('query_labels.npy')
# list_query_label

In [5]:
# n_retrievals = len(retrieval_files)
# retrieval_matrix = np.zeros((n_retrievals, 768))
# for i in tqdm(range(n_retrievals)):
#     try:
#         img = Image.open(retrieval_path+retrieval_files[i]).convert('RGB')
#         vector = model(transforms(img).unsqueeze(0)).detach().numpy()
#         retrieval_matrix[i, :] = vector
#     except:
#         retrieval_matrix[i, :] = np.zeros((1, 768))
# np.save('EVA_retrieval_matrix.npy', retrieval_matrix)
# np.save('retrieval_labels.npy', retrieval_files)
# print(retrieval_matrix.shape)
# retrieval_matrix

In [4]:
# matrix = np.load('EVA_retrieval_matrix.npy')
# # print(np.where(matrix==np.zeros((1, 768))))
# # matrix[9695:9699, :]
# np.sum(np.all(matrix == np.zeros((1, 768)), axis=1))
'''
ERROR INDEX
(9, [2749, 3952, 4893, 5296, 6288, 8028, 8463, 8735, 9697])
'''

9

In [52]:
def calc_Euclid_dist(B1, B2):
    # B1 = query_matrix
    # B2 = retrieval_matrix
    distances = np.zeros((B1.shape[0], B2.shape[0]))
    for i in range(200):
        distances[i, :] = np.sum((B1[i] - B2)**2, axis = 1)
    distances = distances**0.5
    return distances
def get_topk(B1, B2, topk): # get topk idx
    ind = np.argsort(calc_Euclid_dist(B1, B2))
    ids = np.array(range(B2.shape[0]))
    ids = ids[ind]
    return ids[:, :topk]
def get_L2(B1, B2, topk): # get topk labels
    ids = list(get_topk(B1, B2, topk))
    topk_labels = retrieval_labels[ids]
    # L2 = [[img_vector_lst[j][0] for j in i] for i in ids] #[[arr[j] for j in i] for i in idx]
    # print(topk_labels)
    return topk_labels

def calc_topk_mAP_2(B1, B2, L1, topk): 
    '''
    B1: query matrix
    B2: retrieval matrix
    L1: labels of query images
    topk: k
    '''
    topk_retrievals = get_L2(B1, B2, topk)
    topk_mAP = 0
    num_query = B1.shape[0]
    # fig, axes = plt.subplots(num_query, topk+1, figsize=(12, 8))
    for iter in tqdm(range(num_query)):
        true_label = L1[iter].split('_')
        true_idx = []
        # axes[iter, 0].imshow(plt.imread('query_images/'+L1[iter]))
        # axes[iter, 0].axis('off')
        for i in range(topk):
            # axes[iter, i+1].imshow(plt.imread('images/'+topk_retrievals[iter][i]))
            # axes[iter, i+1].axis('off')
            if topk_retrievals[iter][i].split('_')[0] == true_label[0]:
                true_idx.append(i+1)
        true_idx = np.array(true_idx)
        # true_idx = (np.asarray(np.where(L2[ids][iter, :]==true_label)) + 1).ravel()
        # print('true_idx', true_idx)
        if true_idx.shape[0] == 0:
            continue
        count = np.linspace(1, true_idx.shape[0], true_idx.shape[0])
        # print('count', count)
        topk_mAP_ = np.mean(count/true_idx)
        # print('topk_mAP_:', topk_mAP_)
        # print('==========================================')
        topk_mAP += topk_mAP_
    # plt.tight_layout()  # Ensure subplots do not overlap
    # plt.show()
    topk_mAP /= num_query
    # print('=====================================================================')
    # print('topk_mAP', topk_mAP)
    return topk_mAP 

In [53]:
query_matrix = np.load('EVA_query_matrix.npy')
query_labels = np.load('query_labels.npy')
retrieval_matrix = np.load('EVA_retrieval_matrix.npy')
retrieval_labels = np.load('retrieval_labels.npy')
query_matrix.shape, query_labels.shape, retrieval_matrix.shape, retrieval_labels.shape


((200, 768), (200,), (10000, 768), (10000,))

In [54]:
error_idx = [2749, 3952, 4893, 5296, 6288, 8028, 8463, 8735, 9697]
retrieval_matrix =  np.delete(retrieval_matrix , error_idx, axis=0) 
retrieval_labels = np.delete(retrieval_labels , error_idx, axis=0) 
retrieval_matrix.shape, retrieval_labels.shape

((9991, 768), (9991,))

In [57]:
topk_lst = [1, 5, 10, 20, 100, 1000,retrieval_labels.shape[0]]
print('EVA mAP: ')
for i in topk_lst:
    print(f'mAP@{i}:', calc_topk_mAP_2(query_matrix, retrieval_matrix, query_labels, i))

EVA mAP: 


100%|██████████████████████████████████████████████| 200/200 [00:00<00:00, 118149.41it/s]


mAP@1: 0.64


100%|███████████████████████████████████████████████| 200/200 [00:00<00:00, 73590.74it/s]


mAP@5: 0.7287499999999999


...

mAP@100: 0.6083424157548639


100%|████████████████████████████████████████████████| 200/200 [00:00<00:00, 2287.31it/s]


mAP@1000: 0.524433552314303


100%|█████████████████████████████████████████████████| 200/200 [00:00<00:00, 238.95it/s]

mAP@9991: 0.4255778898918473
